In [ ]:
# paketler
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch, cm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase.pdfmetrics import stringWidth

from pandas import read_excel
from PyPDF2 import PdfFileWriter, PdfFileReader

import copy
import io

In [ ]:
# fontları yükle
pdfmetrics.registerFont(TTFont('Montserrat-Bold',"fonts\\Montserrat-Bold.ttf"))
pdfmetrics.registerFont(TTFont('Montserrat-Regular',"fonts\\Montserrat-Regular.ttf"))
pdfmetrics.registerFont(TTFont('Montserrat-Light',"fonts\\Montserrat-Light.ttf"))
pdfmetrics.registerFont(TTFont('Montserrat-LightItalic',"fonts\\Montserrat-LightItalic.ttf"))
pdfmetrics.registerFont(TTFont('Montserrat-Black',"fonts\\Montserrat-Black.ttf"))
pdfmetrics.registerFont(TTFont('NeueBold',"fonts\\NeueBold.ttf"))
pdfmetrics.registerFont(TTFont('HelveticaNeueLight',"fonts\\HelveticaNeueLight.ttf"))
pdfmetrics.registerFont(TTFont('HelveticaNeueLightItalic',"fonts\\HelveticaNeueLightItalic.ttf"))

In [ ]:
# dosyayı al.
katilimci_listesi = read_excel('yaka_karti_listesi.xlsx',sheet_name="yaka_kartlari")
katilimci_listesi

In [ ]:
symposium = "IMSS'19"
line1 = "In recognition of a substantial contribution to the "
line2 = "through participation as "

packet = io.BytesIO()

kart_boyutu = (11.69*inch, 8.27*inch)
kanvas = canvas.Canvas(packet, pagesize=kart_boyutu)

for index, row in katilimci_listesi.iterrows():
    x = 0.85*inch
    y = 3.30*inch
    isim, katilimci_tipi = row['Name'], row['Type']
    
    # ismi yazdır
    kanvas.setFont("Montserrat-Bold", 20)
    kanvas.drawString(0.85*inch, 3.8*inch, isim)
    
    # line1 yazdır
    kanvas.setFont('Montserrat-Light',16)
    kanvas.drawString(x,y,line1)
    
    # sempozyum adını yazdır
    textWidth = stringWidth(line1, 'Montserrat-Light', 16)
    x += textWidth + 1
    kanvas.setFont("Montserrat-Bold", 16)
    kanvas.drawString(x,y,symposium)
    
    # line2 yazdır
    y-=0.3*inch
    x = 0.85*inch
    kanvas.setFont('Montserrat-Light',16)
    kanvas.drawString(x,y,line2)
    
    # katilimci tipini yazdır
    textWidth = stringWidth(line2, 'Montserrat-Light', 16)
    x += textWidth + 1
    kanvas.setFont("Montserrat-Bold", 16)
    kanvas.drawString(x,y,katilimci_tipi+".")

    # sayfayı bitir
    kanvas.showPage()

kanvas.save()

In [ ]:
# sayfanın başına gel
packet.seek(0)

# az önce oluşturduğumuz pdfyi kaydet
canvas_pdf = PdfFileReader(packet)
# boş sertifika pdfyi al
bos_sertifika = PdfFileReader(open("bos_sertifika.pdf", "rb"))

# birleştirilecek pdf yi hazırla
output = PdfFileWriter()

# canvas pdf'deki herbir yazıyı boş pdf ile birleştir ve yeni pdf yi kaydet
for i in range(0,canvas_pdf.getNumPages()):
    page = copy.copy(bos_sertifika.getPage(0))
    page.mergePage(canvas_pdf.getPage(i))
    output.addPage(page)
    print("{}. sertifika oluşturuldu.".format(i))
    
with open("sertifikalar.pdf", "wb") as outfile:
    output.write(outfile)
